### Run the code below if pyspark operations throw random errors

In [ ]:
import shutil
shutil.rmtree("C:/Users/m/AppData/Local/Temp", ignore_errors=True) 

### Importing libraries, starting spark session and taking a look at the data

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
try:
    existing_spark = SparkSession.getActiveSession()
    if existing_spark:
        existing_spark.stop()
        print("Previous SparkSession stopped.")
except Exception as e:
    print(f"Could not get and stop the active SparkSession: {e}")
    
spark = SparkSession.builder.appName("Finance_Model_Training").getOrCreate()

In [4]:
train_df = spark.read.csv("data/Train_data.csv", header=True, inferSchema=True)
test_df = spark.read.csv("data/Train_data.csv", header=True, inferSchema=True)

In [5]:
train_df.show()

+--------+-------------+----------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|   service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|d

In [6]:
train_df.columns

['duration',
 'protocol_type',
 'service',
 'flag',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'class']

### To understand what features we are playing with, description of all the features is given below:

##### **Basic features which describe the connection without looking at the payload**
- **duration** – Length (in seconds) of the connection.
- **protocol_type** – Network protocol used (e.g., TCP, UDP, ICMP).
- **service** – Destination service (e.g., http, telnet, ftp).
- **flag** – Status flag of the connection (e.g., S0, SF, REJ).
- **src_bytes** – Number of data bytes sent from source to destination.
- **dst_bytes** – Number of data bytes sent from destination to source.
- **land** – 1 if connection is from/to the same host/port; 0 otherwise.
- **wrong_fragment** – Number of wrong fragments in the packet.
- **urgent** – Number of urgent packets.
##### **Content features which describe what is within the connection payload**
- **hot** – Number of “hot” indicators (e.g., suspicious commands).
- **num_failed_logins** – Number of failed login attempts.
- **logged_in** – 1 if successfully logged in; 0 otherwise.
- **num_compromised** – Number of compromised conditions.
- **root_shell** – 1 if root shell is obtained; 0 otherwise.
- **su_attempted** – 1 if “su root” command attempted; 0 otherwise.
- **num_root** – Number of “root” accesses.
- **num_file_creations** – Number of file creation operations.
- **num_shells** – Number of shell prompts opened.
- **num_access_files** – Number of attempts to access control files.
- **num_outbound_cmds** – Number of outbound commands.
- **is_host_login** – 1 if user is a “host login”; 0 otherwise.
- **is_guest_login** – 1 if user is a “guest login”; 0 otherwise.
##### **Traffic features based on the past 2 seconds wiindow for same host
- **count** – Number of connections to the same host in the past 2 seconds.
- **srv_count** – Number of connections to the same service.
- **serror_rate** – % of connections with “SYN” errors.
- **srv_serror_rate** – % of connections with “SYN” errors to the same service.
- **rerror_rate** – % of connections with “REJ” errors.
- **srv_rerror_rate** – % of REJ errors to the same service.
- **same_srv_rate** – % of connections to the same service.
- **diff_srv_rate** – % of connections to different services.
- **srv_diff_host_rate** – % of connections to the same service but different host.
- **dst_host_count** – Number of connections to the destination host.
- **dst_host_srv_count** – Connections to the destination host using the same service.
- **dst_host_same_srv_rate** – % of same-service connections among dst_host_count.
- **dst_host_diff_srv_rate** – % of different-service connections.
- **dst_host_same_src_port_rate** – % of connections from same source port.
- **dst_host_srv_diff_host_rate** – % of same-service connections to different hosts.
- **dst_host_serror_rate** – % of connections with SYN errors.
- **dst_host_srv_serror_rate** – % of same-service SYN errors.
- **dst_host_rerror_rate** – % of connections with REJ errors.
- **dst_host_srv_rerror_rate** – % of same-service REJ errors.
##### **Label**
- **class** - either "normal" or "anomally"

#### Let's see if the data has duplicates and/or missing values

In [7]:
if train_df.count() == train_df.na.drop(how="any").count():
    print("No missing values in the dataset")
else:
    print("Missing values, need to treat the dataset")

if train_df.count() == train_df.dropDuplicates().count():
    print("No duplicates in the dataset")
else:
    print("Duplicate values, need to treat the dataset")

No missing values in the dataset
No duplicates in the dataset


Lets check the data distribution for all the columns

In [8]:
train_df.describe().show()

+-------+------------------+-------------+-------+-----+------------------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+------------------+--------------------+--------------------+--------------------+-----------------+-------------+--------------------+------------------+-----------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+------------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|summary|          duration|protocol_type|service| flag|         src_bytes|         dst_bytes|                land|      wrong_fragment|              urgent|           

You can notice that some of the columns min, max, average and median is the same value, hence they don't have any value variation. Dropping these columns will help make our training more efficient

In [9]:
train_df = train_df.drop("num_outbound_cmds", "is_host_login")

We will not be checking for outliers in the data as they are valuable for the potential anomally detection i.e. financial fraud. We will look into class imbalance and resolve it further

In [10]:
train_df.groupBy().pivot("class").count().show()

+-------+------+
|anomaly|normal|
+-------+------+
|  11743| 13449|
+-------+------+



The classes are in the ratio of 45:55 which is decently balanced, hence we don't need to do further balancing in such scenario. Lets check if the datatypes are assigned accurately or if they need to be changed

In [11]:
train_df.show()

+--------+-------------+----------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|   service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_host

In [12]:
train_df.printSchema()

root
 |-- duration: integer (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: integer (nullable = true)
 |-- dst_bytes: integer (nullable = true)
 |-- land: integer (nullable = true)
 |-- wrong_fragment: integer (nullable = true)
 |-- urgent: integer (nullable = true)
 |-- hot: integer (nullable = true)
 |-- num_failed_logins: integer (nullable = true)
 |-- logged_in: integer (nullable = true)
 |-- num_compromised: integer (nullable = true)
 |-- root_shell: integer (nullable = true)
 |-- su_attempted: integer (nullable = true)
 |-- num_root: integer (nullable = true)
 |-- num_file_creations: integer (nullable = true)
 |-- num_shells: integer (nullable = true)
 |-- num_access_files: integer (nullable = true)
 |-- is_guest_login: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- srv_count: integer (nullable = true)
 |-- serror_rate: double (nullable = true)
 |-- srv_

#### The columns look fine and the correct values seem to be assigned. Lets encode all the string columns to indexed columns and then perform one hot encoding, so its readable to our ML model

In [13]:
# Getting all the column names which are string
columnList = [item[0] for item in train_df.dtypes if item[1].startswith('string')]

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

index_cols = [c + "_index" for c in columnList]
ohe_cols = [c + "_ohe" for c in columnList]

# Step 1: Index string columns to numeric
indexers = [StringIndexer(inputCol=c, outputCol=c + "_index") for c in columnList]

# Step 2: OneHot encode the indexed columns
ohe = OneHotEncoder(inputCols=index_cols, outputCols=ohe_cols)

# Step 3: Build pipeline
pipeline = Pipeline(stages=indexers + [ohe])

# Fit and transform
encoder_model = pipeline.fit(train_df)
train_encoded_df = encoder_model.transform(train_df)
test_encoded_df = encoder_model.transform(test_df)

# To show the change from original value, indexed value and one hot encoded value
train_encoded_df.select(columnList + index_cols + ohe_cols).show()

index_cols.remove("class_index")
train_encoded_df = train_encoded_df.drop(*(index_cols))
test_encoded_df = train_encoded_df.drop(*(index_cols))

+-------------+----------+----+-------+-------------------+-------------+----------+-----------+-----------------+---------------+--------------+-------------+
|protocol_type|   service|flag|  class|protocol_type_index|service_index|flag_index|class_index|protocol_type_ohe|    service_ohe|      flag_ohe|    class_ohe|
+-------------+----------+----+-------+-------------------+-------------+----------+-----------+-----------------+---------------+--------------+-------------+
|          tcp|  ftp_data|  SF| normal|                0.0|          4.0|       0.0|        0.0|    (2,[0],[1.0])| (65,[4],[1.0])|(10,[0],[1.0])|(1,[0],[1.0])|
|          udp|     other|  SF| normal|                1.0|          6.0|       0.0|        0.0|    (2,[1],[1.0])| (65,[6],[1.0])|(10,[0],[1.0])|(1,[0],[1.0])|
|          tcp|   private|  S0|anomaly|                0.0|          1.0|       1.0|        1.0|    (2,[0],[1.0])| (65,[1],[1.0])|(10,[1],[1.0])|    (1,[],[])|
|          tcp|      http|  SF| normal| 

You may notice that the one hot encoded values when ".show()" is used don't look like arrays. However, they are arrays. To understand what it means, I will use an example given below:

(65,[44],[1.0]) => the array has 65 values and the index 44 has a non-zero value i.e. 1.0

Alright, we have done plenty of data preprocessing, we can move on to normalizing the data but before that, we have to use Vector Assembler to combine input columns into single vector column

In [15]:
# Getting the list of input columns
input_cols = train_encoded_df.columns
for i in columnList:
    input_cols.remove(i)
input_cols.remove("class_index")
input_cols.remove("class_ohe")

In [16]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
assembler = VectorAssembler(inputCols=input_cols, outputCol="features_vector")
train_assembled_df = assembler.transform(train_encoded_df)
test_assembled_df = assembler.transform(test_encoded_df)

vec_assembler = VectorAssembler(
    inputCols=["class_ohe"],  # or any other OHE columns
    outputCol="class_ohe_vec"
)
train_assembled_df = vec_assembler.transform(train_assembled_df)
test_assembled_df = vec_assembler.transform(test_assembled_df)

In [17]:
scaler = MinMaxScaler(inputCol="features_vector", outputCol="scaled_features")
scaler_model = scaler.fit(train_assembled_df)
train_scaled_df = scaler_model.transform(train_assembled_df)
test_scaled_df = scaler_model.transform(test_assembled_df)

In [18]:
train_scaled_df.show()

+--------+-------------+----------+----+---------+---------+----+--------------+------+---+-----------------+---------+---------------+----------+------------+--------+------------------+----------+----------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+-----------+-----------------+---------------+--------------+-------------+--------------------+-------------+--------------------+
|duration|protocol_type|   service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_ra

## Model Training

### Random forest + Filter Method

In [22]:
train = train_scaled_df.select(["scaled_features", "class_index"])
test = test_scaled_df.select(["scaled_features", "class_index"])

In [23]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=100, maxDepth=10, seed=42)
rf_model = rf.fit(train)

In [ ]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9955541441727532


The number of trees and depth may be too high, let's reduce it to make the model less heavy. We know we are not overfitting as the model performs well on unseen data (training data).

In [ ]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=40, maxDepth=7, seed=42)
rf_model = rf.fit(train)

In [ ]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9897983486821214
AUC :  0.9996666753349758
F1 score :  0.9898400765595308


Let's reduce it more and see the performance

In [ ]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=30, maxDepth=5, seed=42)
rf_model = rf.fit(train)

In [ ]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9761432200698634
AUC :  0.9981615333021976
F1 score :  0.9763797495972262


Depth = 5 seems more appropriate, as the performance didn't drop as much and the complexity of the model has significantly dropped. Lets check the feature importance and discard all the features which are not significantly contributing to the model

In [ ]:
rf_model.featureImportances

SparseVector(113, {0: 0.0006, 1: 0.1054, 2: 0.1128, 3: 0.0, 4: 0.0006, 5: 0.0, 6: 0.004, 7: 0.0, 8: 0.0499, 9: 0.0048, 12: 0.0, 13: 0.0, 15: 0.0, 16: 0.0002, 17: 0.0458, 18: 0.0072, 19: 0.0321, 20: 0.0065, 21: 0.0056, 22: 0.0083, 23: 0.0791, 24: 0.0241, 25: 0.001, 26: 0.0091, 27: 0.1287, 28: 0.0263, 29: 0.0548, 30: 0.0326, 31: 0.0182, 32: 0.0034, 33: 0.0049, 34: 0.0022, 35: 0.006, 36: 0.0118, 37: 0.0023, 38: 0.0038, 39: 0.009, 40: 0.0218, 41: 0.0024, 42: 0.0012, 43: 0.0074, 44: 0.0005, 45: 0.0137, 46: 0.0001, 47: 0.0005, 48: 0.0002, 49: 0.0001, 53: 0.0, 60: 0.0001, 68: 0.0, 74: 0.0, 87: 0.0, 88: 0.0, 91: 0.0001, 100: 0.0001, 103: 0.0872, 104: 0.0586, 105: 0.0003, 106: 0.0041, 107: 0.0, 108: 0.0001, 109: 0.0001, 111: 0.0})

In [ ]:
threshold = 0.01  # Adjust as needed
low_importance_indices = [i for i, score in enumerate(rf_model.featureImportances) if score < threshold]

In [ ]:
len(low_importance_indices)

96

In [ ]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.sql.functions import udf

def drop_indices_from_vector(vector, drop_indices):
    if not isinstance(vector, SparseVector):
        vector = Vectors.sparse(len(vector), [(i, v) for i, v in enumerate(vector)])
    
    kept_items = [(i, v) for i, v in zip(vector.indices, vector.values) if i not in drop_indices]
    new_indices, new_values = zip(*kept_items) if kept_items else ([], [])
    return SparseVector(len(vector), new_indices, new_values)

drop_udf = udf(lambda vec: drop_indices_from_vector(vec, low_importance_indices))

In [ ]:
train_reduced_df = train_scaled_df.withColumn("features_reduced", drop_udf("scaled_features"))
test_reduced_df = test_scaled_df.withColumn("features_reduced", drop_udf("scaled_features"))

In [ ]:
train = train_reduced_df.select(["scaled_features", "class_index"])
test = test_reduced_df.select(["scaled_features", "class_index"])

In [ ]:
rf = RandomForestClassifier(featuresCol="scaled_features", labelCol="class_index",numTrees=30, maxDepth=5, seed=42)
rf_model = rf.fit(train)

In [ ]:
results = rf_model.evaluate(test)

prec = results.weightedPrecision
reca = results.weightedRecall
f1 = 2*prec*reca/(prec+reca)

print("Accuracy : ", results.accuracy)
print("AUC : ", results.areaUnderROC)
print("F1 score : ", f1)

Accuracy :  0.9761432200698634
AUC :  0.9981615333021976
F1 score :  0.9763797495972262


In [ ]:
rf_model.save("model/random_forest")

Py4JJavaError: An error occurred while calling o3299.save.
: java.io.IOException: Path model/random_forest already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand